In [1]:
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
from kornia import color
import torch
import pandas as pd
import torchvision

/opt/conda/envs/perturbations/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


https://cvi2.uni.lu/spark-2021/

# Processing

## Crop and Rescale

In [3]:
val_labels = pd.read_csv("/data/ICIP-2021/validate/validate_labels.csv")
train_labels = pd.read_csv("/data/ICIP-2021/train/train_labels.csv")

In [4]:
val_labels["bbox"] = val_labels["bbox"].apply(lambda x: [int(y) for y in x[1:-1].split(",")])
train_labels["bbox"] = train_labels["bbox"].apply(lambda x: [int(y) for y in x[1:-1].split(",")])

In [5]:
val_labels

,class,id,image,depth,bbox,illumination
0,Cloudsat,10563,image_10563_img.png,image_10563_depth.png,"[262, 283, 323, 367]",81.036141
1,Terra,6536,image_06536_img.png,image_06536_depth.png,"[476, 469, 542, 497]",0.244364
2,Sentinel-6,4676,image_04676_img.png,image_04676_depth.png,"[520, 445, 584, 516]",0.126758
3,Sentinel-6,2965,image_02965_img.png,image_02965_depth.png,"[377, 606, 692, 945]",0.686202
4,CubeSat,3907,image_03907_img.png,image_03907_depth.png,"[523, 421, 560, 464]",0.480803
...,...,...,...,...,...,...
29995,CubeSat,4350,image_04350_img.png,image_04350_depth.png,"[714, 573, 761, 606]",0.582582
29996,CubeSat,7779,image_07779_img.png,image_07779_depth.png,"[360, 657, 381, 681]",0.368994
29997,CubeSat,5426,image_05426_img.png,image_05426_depth.png,"[537, 737, 567, 783]",0.350740
29998,Cloudsat,6952,image_06952_img.png,image_06952_depth.png,"[420, 402, 466, 449]",34.593478


In [5]:
train_labels

,class,id,image,depth,bbox,illumination
0,CubeSat,10299,image_10299_img.png,image_10299_depth.png,"[296, 536, 353, 581]",0.126822
1,Jason,12016,image_12016_img.png,image_12016_depth.png,"[580, 434, 712, 753]",0.508580
2,Debris,6332,image_06332_img.png,image_06332_depth.png,"[437, 406, 503, 463]",0.103017
3,TRMM,2173,image_02173_img.png,image_02173_depth.png,"[0, 151, 312, 538]",0.883734
4,Cloudsat,11207,image_11207_img.png,image_11207_depth.png,"[450, 425, 509, 481]",3.576220
...,...,...,...,...,...,...
89995,Debris,24362,image_24362_img.png,image_24362_depth.png,"[447, 440, 501, 478]",53.173303
89996,Sentinel-6,8834,image_08834_img.png,image_08834_depth.png,"[291, 209, 485, 407]",0.728906
89997,Cloudsat,226,image_00226_img.png,image_00226_depth.png,"[506, 472, 556, 519]",88.962500
89998,Jason,11172,image_11172_img.png,image_11172_depth.png,"[422, 438, 515, 631]",36.451119


In [6]:
labels = train_labels["class"].unique()
labels.sort()
label_map = {x: i for i, x in enumerate(labels)}

In [7]:
train_labels["class_map"] = train_labels["class"].apply(lambda x: label_map[x])
val_labels["class_map"] = val_labels["class"].apply(lambda x: label_map[x])

In [8]:
val_labels

,class,id,image,depth,bbox,illumination,class_map
0,Cloudsat,10563,image_10563_img.png,image_10563_depth.png,"[262, 283, 323, 367]",81.036141,4
1,Terra,6536,image_06536_img.png,image_06536_depth.png,"[476, 469, 542, 497]",0.244364,10
2,Sentinel-6,4676,image_04676_img.png,image_04676_depth.png,"[520, 445, 584, 516]",0.126758,8
3,Sentinel-6,2965,image_02965_img.png,image_02965_depth.png,"[377, 606, 692, 945]",0.686202,8
4,CubeSat,3907,image_03907_img.png,image_03907_depth.png,"[523, 421, 560, 464]",0.480803,5
...,...,...,...,...,...,...,...
29995,CubeSat,4350,image_04350_img.png,image_04350_depth.png,"[714, 573, 761, 606]",0.582582,5
29996,CubeSat,7779,image_07779_img.png,image_07779_depth.png,"[360, 657, 381, 681]",0.368994,5
29997,CubeSat,5426,image_05426_img.png,image_05426_depth.png,"[537, 737, 567, 783]",0.350740,5
29998,Cloudsat,6952,image_06952_img.png,image_06952_depth.png,"[420, 402, 466, 449]",34.593478,4


In [8]:
np.save("/data/ICIP-2021/processed3/train.npy", train_labels["class_map"].values)
np.save("/data/ICIP-2021/processed3/val.npy", val_labels["class_map"].values)

In [31]:
rgbs = []
depths = []
resize = torchvision.transforms.Resize((512, 512))
togs = color.RgbToGrayscale()
toluv = color.RgbToLuv()

for i, (class_, rgb_name, depth_name, bbox) in enumerate(zip(train_labels["class"], train_labels["image"], train_labels["depth"], train_labels["bbox"])):
    rgb = Image.open(f"/data/ICIP-2021/train/train_rgb/{class_}/{rgb_name}")

    rgb = torch.tensor(np.array(rgb), dtype=torch.float).moveaxis(2, 0)

    xmin, ymin, xmax, ymax = bbox[0], bbox[1], bbox[2], bbox[3]
    rgb = resize(rgb[:, xmin:xmax, ymin:ymax]) / 255

    np.save(f"/data/ICIP-2021/processed3/train/{str(i).zfill(6)}.npy", rgb.numpy())

In [32]:
resize = torchvision.transforms.Resize((512, 512))
togs = color.RgbToGrayscale()
toluv = color.RgbToLuv()

for i, (class_, rgb_name, depth_name, bbox) in enumerate(zip(val_labels["class"], val_labels["image"], val_labels["depth"], val_labels["bbox"])):
    rgb = Image.open(f"/data/ICIP-2021/validate/validate_rgb/{class_}/{rgb_name}")

    rgb = torch.tensor(np.array(rgb), dtype=torch.float).moveaxis(2, 0)

    xmin, ymin, xmax, ymax = bbox[0], bbox[1], bbox[2], bbox[3]
    rgb = resize(rgb[:, xmin:xmax, ymin:ymax]) / 255

    np.save(f"/data/ICIP-2021/processed3/val/{str(i).zfill(6)}.npy", rgb.numpy())

## Mean and Std

In [33]:
mean = np.zeros(3)
std = np.zeros(3)
for i in range(len(train_labels["bbox"])):
    img = np.load(f"/data/ICIP-2021/processed3/train/{str(i).zfill(6)}.npy")
    mean += np.mean(img, axis=(1, 2))
    std += np.std(img, axis=(1, 2))

In [34]:
mean / len(train_labels["bbox"]), std / len(train_labels["bbox"])

(array([0.09430549, 0.09299065, 0.09526419]),
 array([0.09868433, 0.09484387, 0.09236576]))

In [164]:
mean = np.zeros(3)
std = np.zeros(3)
for i in range(len(val_labels["bbox"])):
    img = np.load(f"/data/ICIP-2021/processed/val/{str(i).zfill(6)}.npy")
    mean += np.mean(img, axis=(1, 2))
    std += np.std(img, axis=(1, 2))

In [165]:
mean / len(val_labels["bbox"]), std / len(val_labels["bbox"])

(array([9.81587756, 0.097757  , 0.61005476]),
 array([9.60466818, 0.09053227, 0.33148834]))

## Same range over channels and compress

In [22]:
mean = np.zeros(3)
std = np.zeros(3)
imgs = []

for i in range(len(train_labels["bbox"])):
    img = np.load(f"/data/ICIP-2021/processed3/train/{str(i).zfill(6)}.npy")
    # img[0] /= 100
    img *= 255
    imgs.append(torchvision.transforms.Resize((32, 32))(torch.from_numpy(img.astype(np.uint8))))

    if (i + 1) % 1000 == 0:
        np.savez_compressed(f"/data/ICIP-2021/processed5/train/{str(i // 1000).zfill(5)}.npz", *imgs)
        del imgs
        imgs = []

In [ ]:
mean = np.zeros(3)
std = np.zeros(3)
for i in range(90):
    imgs = np.load(f"/data/ICIP-2021/processed2/train/{str(i).zfill(5)}.npz")
    for ii in range(1000):
        img = getattr(imgs.f, f"arr_{ii}").astype(np.float32) / 255
        mean += np.mean(img, axis=(1, 2))
        std += np.std(img, axis=(1, 2))

In [6]:
mean / len(train_labels["bbox"]), std / len(train_labels["bbox"])

(array([0.09244801, 0.09198833, 0.60900891]),
 array([0.10071791, 0.09504412, 0.33186111]))

In [25]:
mean = np.zeros(3)
std = np.zeros(3)
imgs = []

for i in range(len(val_labels["bbox"])):
    img = np.load(f"/data/ICIP-2021/processed3/val/{str(i).zfill(6)}.npy")
    # img[0] /= 100
    img *= 255
    imgs.append(torchvision.transforms.Resize((32, 32))(torch.from_numpy(img.astype(np.uint8))))

    if (i + 1) % 1000 == 0:
        np.savez_compressed(f"/data/ICIP-2021/processed5/val/{str(i // 1000).zfill(5)}.npz", *imgs)
        del imgs
        imgs = []

In [124]:
mean = np.zeros(3)
std = np.zeros(3)
for i in range(30):
    imgs = np.load(f"/data/ICIP-2021/processed2/val/{str(i).zfill(5)}.npz")
    for ii in range(1000):
        img = getattr(imgs.f, f"arr_{ii}").astype(np.float32) / 255
        
        mean += np.mean(img, axis=(1, 2))
        std += np.std(img, axis=(1, 2))

In [125]:
mean / len(val_labels["bbox"]), std / len(val_labels["bbox"])

(array([0.09638784, 0.09602991, 0.60852957]),
 array([0.09595593, 0.09040637, 0.33188107]))